In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 73)

In [3]:
game_df = pd.read_pickle('data/games.pickle')

In [4]:
game_df.shape

(20508, 38)

In [5]:
game_df.columns

Index(['season_year', 'week_num', 'full_game_date', 'team', 'opp',
       'game_day_of_week', 'game_date', 'game_outcome', 'pts_off', 'pts_def',
       'pass_cmp', 'pass_att', 'pass_yds', 'pass_td', 'pass_int',
       'pass_sacked', 'pass_sacked_yds', 'pass_yds_per_att',
       'pass_net_yds_per_att', 'pass_cmp_perc', 'pass_rating', 'rush_att',
       'rush_yds', 'rush_yds_per_att', 'rush_td', 'fgm', 'fga', 'xpm', 'xpa',
       'punt', 'punt_yds', 'overtime', 'game_location', 'time_of_poss',
       'third_down_success', 'third_down_att', 'fourth_down_success',
       'fourth_down_att'],
      dtype='object')

In [6]:
mask = (game_df.team == 'chi') & (game_df.season_year == 2019)
game_df[mask]

,season_year,week_num,full_game_date,team,opp,game_day_of_week,game_date,game_outcome,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,game_location,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att
19520,2019,1.0,2019-09-05,chi,gnb,Thu,September 5,L,3.0,10.0,26.0,45.0,208.0,0.0,1.0,5.0,20.0,5.0,4.0,57.0,60.0,15.0,46.0,3.0,0.0,1.0,1.0,0.0,0.0,8.0,341.0,NaN,NaN,28:57,3.0,15.0,0.0,2.0
19521,2019,2.0,2019-09-15,chi,den,Sun,September 15,W,16.0,14.0,16.0,27.0,120.0,0.0,0.0,0.0,0.0,4.0,4.0,59.0,70.0,29.0,153.0,5.0,1.0,3.0,3.0,1.0,1.0,5.0,287.0,NaN,@,27:01,3.0,11.0,1.0,1.0
19522,2019,3.0,2019-09-23,chi,was,Mon,September 23,W,31.0,15.0,25.0,31.0,208.0,3.0,1.0,3.0,23.0,7.0,6.0,80.0,113.0,24.0,90.0,3.0,0.0,1.0,2.0,4.0,4.0,3.0,140.0,NaN,@,31:36,8.0,13.0,0.0,0.0
19523,2019,4.0,2019-09-29,chi,min,Sun,September 29,W,16.0,6.0,24.0,33.0,197.0,1.0,0.0,1.0,7.0,6.0,5.0,72.0,97.0,33.0,72.0,2.0,0.0,3.0,3.0,1.0,1.0,5.0,241.0,NaN,NaN,35:27,5.0,16.0,2.0,3.0
19524,2019,5.0,2019-10-06,chi,rai,Sun,October 6,L,21.0,24.0,22.0,30.0,194.0,2.0,2.0,4.0,37.0,7.0,5.0,73.0,84.0,17.0,42.0,2.0,1.0,0.0,0.0,3.0,3.0,5.0,213.0,NaN,@,25:17,4.0,11.0,0.0,0.0
19525,2019,7.0,2019-10-20,chi,nor,Sun,October 20,L,25.0,36.0,34.0,54.0,235.0,2.0,0.0,2.0,16.0,4.0,4.0,63.0,85.0,7.0,17.0,2.0,0.0,1.0,1.0,2.0,2.0,7.0,263.0,NaN,NaN,22:34,2.0,12.0,1.0,2.0
19526,2019,8.0,2019-10-27,chi,sdg,Sun,October 27,L,16.0,17.0,23.0,35.0,226.0,0.0,1.0,4.0,27.0,7.0,5.0,65.0,71.0,38.0,162.0,4.0,1.0,3.0,5.0,1.0,1.0,3.0,123.0,NaN,NaN,38:00,5.0,14.0,1.0,1.0
19527,2019,9.0,2019-11-03,chi,phi,Sun,November 3,L,14.0,22.0,10.0,21.0,102.0,0.0,0.0,3.0,23.0,6.0,4.0,47.0,62.0,18.0,62.0,3.0,2.0,0.0,0.0,2.0,2.0,8.0,372.0,NaN,@,19:42,2.0,10.0,0.0,0.0
19528,2019,10.0,2019-11-10,chi,det,Sun,November 10,W,20.0,13.0,16.0,23.0,145.0,3.0,0.0,5.0,28.0,7.0,5.0,69.0,125.0,24.0,81.0,3.0,0.0,0.0,0.0,2.0,3.0,9.0,387.0,NaN,NaN,28:06,2.0,12.0,1.0,1.0
19529,2019,11.0,2019-11-17,chi,ram,Sun,November 17,L,7.0,17.0,25.0,47.0,193.0,1.0,1.0,3.0,6.0,4.0,3.0,53.0,61.0,24.0,74.0,3.0,0.0,0.0,2.0,1.0,1.0,6.0,258.0,NaN,@,32:44,6.0,17.0,0.0,2.0


In [7]:
# clean the game_location column & apply change. def home_game(row):
def home_game(row):    
    if row == '@':
        return 0
    return 1

game_df['home'] = game_df.game_location.apply(home_game)
game_df.drop("game_location", axis=1, errors='ignore', inplace=True)

In [8]:
# add team_year identifier to each row (to easily groupby team/year)

team_years = game_df['team'] + '-' +  game_df['season_year'].astype(str) 
game_df.insert(loc=2, column='team_year', value=team_years)
game_df.sample()

,season_year,week_num,team_year,full_game_date,team,opp,game_day_of_week,game_date,game_outcome,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,home
13247,2007,8.0,clt-2007,2007-10-28,clt,car,Sun,October 28,W,31.0,7.0,15.0,32.0,264.0,2.0,0.0,0.0,0.0,8.0,8.0,46.0,96.0,32.0,131.0,4.0,2.0,1.0,1.0,4.0,4.0,4.0,165.0,NaN,23:55,6.0,12.0,0.0,1.0,0


Datetime formatting strings:

- ``%B`` month fullname
- ``%-d`` day of month without leading zero\*
    - \*note: the ``-`` only works on Linux/OS-X...for windows replace with '#'
- ``%Y`` full year

In [9]:
# convert game date to datetime & add new column to DF


# f = r"%B %d-%Y"
# full_game_date = game_df.game_date + '-' + game_df.year.astype(str)
# full_game_date = pd.to_datetime(full_game_date, format=f)
# game_df.insert(loc=2, column='date', value=full_game_date)
# game_df.sample(3)

In [10]:
# now drop the game_date col
game_df.drop("game_date", axis=1, inplace=True, errors='ignore')
game_df.sample(2) # to confirm

,season_year,week_num,team_year,full_game_date,team,opp,game_day_of_week,game_outcome,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,home
9379,2000,5.0,buf-2000,2000-10-01,buf,clt,Sun,L,16.0,18.0,21.0,32.0,215.0,1.0,0.0,4.0,31.0,7.0,6.0,65.0,95.0,33.0,170.0,5.0,0.0,3.0,3.0,1.0,1.0,5.0,195.0,NaN,35:59,7.0,16.0,1.0,1.0,1
16056,2012,5.0,nyj-2012,2012-10-08,nyj,htx,Mon,L,17.0,23.0,14.0,32.0,217.0,1.0,2.0,3.0,13.0,7.0,6.0,43.0,51.0,23.0,69.0,3.0,0.0,1.0,1.0,2.0,2.0,6.0,265.0,NaN,24:44,4.0,14.0,2.0,2.0,1


In [11]:
# add decade column
decades = game_df['season_year'] // 10
game_df.insert(loc=5, column='decade', value=decades)
game_df[['full_game_date', 'decade']].sample(10)

,full_game_date,decade
3552,1988-10-02,198
18419,2017-12-31,201
15751,2013-01-12,201
9785,2000-11-20,200
15925,2012-10-21,201
1538,1983-12-04,198
18063,2016-11-20,201
12930,2006-12-31,200
19409,2018-09-30,201
13292,2007-09-09,200


In [12]:
# add unique game identifier that will be the same for both teams in the same game


def apply_game_id(row):
    teams = []
    teams.append(str(row['team']))
    teams.append(str(row['opp']))
    teams.sort()
    
    game_id = teams[0] + '-' +  teams[1] + '-' + str(row['full_game_date'])[:-9]
    return game_id

In [13]:
game_ids = game_df.apply(apply_game_id, axis=1)
game_df.insert(loc=0, column='game_id', value=game_ids)

In [14]:
game_df

,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,home
0,crd-nyg-1980-09-07,1980,1.0,crd-1980,1980-09-07,crd,198,nyg,Sun,L,35.0,41.0,18.0,31.0,287.0,2.0,3.0,4.0,35.0,10.0,8.0,58.0,71.0,24.0,132.0,5.0,3.0,0.0,0.0,5.0,5.0,4.0,167.0,NaN,NaN,NaN,NaN,NaN,NaN,1
1,crd-sfo-1980-09-14,1980,2.0,crd-1980,1980-09-14,crd,198,sfo,Sun,L,21.0,24.0,28.0,38.0,256.0,2.0,1.0,2.0,9.0,7.0,6.0,73.0,98.0,37.0,86.0,2.0,1.0,0.0,1.0,3.0,3.0,7.0,253.0,OT,NaN,NaN,NaN,NaN,NaN,0
2,crd-det-1980-09-21,1980,3.0,crd-1980,1980-09-21,crd,198,det,Sun,L,7.0,20.0,22.0,36.0,242.0,1.0,1.0,5.0,33.0,7.0,5.0,61.0,78.0,18.0,82.0,4.0,0.0,0.0,1.0,1.0,1.0,6.0,275.0,NaN,NaN,NaN,NaN,NaN,NaN,0
3,crd-phi-1980-09-28,1980,4.0,crd-1980,1980-09-28,crd,198,phi,Sun,W,24.0,14.0,11.0,21.0,108.0,0.0,0.0,0.0,0.0,5.0,5.0,52.0,67.0,41.0,195.0,4.0,2.0,1.0,2.0,3.0,3.0,5.0,205.0,NaN,NaN,NaN,NaN,NaN,NaN,1
4,crd-nor-1980-10-05,1980,5.0,crd-1980,1980-10-05,crd,198,nor,Sun,W,40.0,7.0,13.0,18.0,103.0,1.0,0.0,1.0,10.0,6.0,5.0,72.0,104.0,62.0,330.0,5.0,4.0,2.0,2.0,4.0,5.0,2.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20503,sfo-was-2020-12-13,2020,14.0,was-2020,2020-12-13,was,202,sfo,Sun,W,23.0,15.0,15.0,32.0,95.0,0.0,1.0,2.0,13.0,3.0,2.0,46.0,40.0,28.0,98.0,3.0,0.0,3.0,4.0,2.0,2.0,8.0,398.0,NaN,27:49,3.0,15.0,0.0,0.0,0
20504,sea-was-2020-12-20,2020,15.0,was-2020,2020-12-20,was,202,sea,Sun,L,15.0,20.0,38.0,55.0,269.0,1.0,2.0,4.0,26.0,5.0,4.0,69.0,70.0,20.0,84.0,4.0,1.0,1.0,1.0,0.0,1.0,4.0,182.0,NaN,32:16,10.0,17.0,0.0,1.0,1
20505,car-was-2020-12-27,2020,16.0,was-2020,2020-12-27,was,202,car,Sun,L,13.0,20.0,26.0,47.0,278.0,1.0,2.0,3.0,13.0,6.0,5.0,55.0,62.0,19.0,108.0,5.0,0.0,2.0,2.0,1.0,1.0,3.0,154.0,NaN,24:24,7.0,15.0,1.0,3.0,1
20506,phi-was-2021-01-03,2020,17.0,was-2020,2021-01-03,was,202,phi,Sun,W,20.0,14.0,22.0,32.0,145.0,2.0,2.0,3.0,17.0,5.0,4.0,68.0,73.0,27.0,103.0,3.0,0.0,2.0,2.0,2.0,2.0,6.0,261.0,NaN,30:48,6.0,15.0,0.0,0.0,0


In [15]:
cat_game_outcome = game_df.loc[:, ['game_outcome']]
cat_game_outcome.value_counts()

game_outcome
L               10229
W               10229
T                  46
dtype: int64

In [16]:
# convert game_outcome col to binary to avoid multicolinearity

game_df[['result_tie', 'result_win']] = pd.get_dummies(game_df.game_outcome, drop_first=True)
game_df.sample(10)

,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,home,result_tie,result_win
18182,nyg-pit-2016-12-04,2016,13.0,nyg-2016,2016-12-04,nyg,201,pit,Sun,L,14.0,24.0,24.0,39.0,178.0,2.0,2.0,2.0,17.0,5.0,4.0,61.0,68.0,14.0,56.0,4.0,0.0,0.0,0.0,2.0,2.0,4.0,177.0,NaN,25:52,4.0,11.0,0.0,3.0,0,0,0
4102,clt-den-1989-10-15,1989,6.0,den-1989,1989-10-15,den,198,clt,Sun,W,14.0,3.0,13.0,25.0,150.0,0.0,1.0,0.0,0.0,6.0,6.0,52.0,53.0,43.0,169.0,3.0,2.0,0.0,0.0,2.0,2.0,5.0,213.0,NaN,37:49,NaN,NaN,NaN,NaN,1,0,1
10908,crd-sea-2003-12-21,2003,16.0,crd-2003,2003-12-21,crd,200,sea,Sun,L,10.0,28.0,25.0,40.0,220.0,1.0,0.0,8.0,54.0,6.0,4.0,62.0,85.0,26.0,79.0,3.0,0.0,1.0,3.0,1.0,1.0,6.0,263.0,NaN,33:14,6.0,18.0,1.0,3.0,0,0,0
1607,nwe-sea-1983-12-18,1983,16.0,sea-1983,1983-12-18,sea,198,nwe,Sun,W,24.0,6.0,13.0,21.0,198.0,2.0,0.0,5.0,32.0,11.0,7.0,61.0,124.0,39.0,170.0,4.0,1.0,1.0,2.0,3.0,3.0,6.0,247.0,NaN,32:34,NaN,NaN,NaN,NaN,1,0,1
12545,buf-nwe-2006-09-10,2006,1.0,buf-2006,2006-09-10,buf,200,nwe,Sun,L,17.0,19.0,15.0,23.0,141.0,0.0,0.0,3.0,23.0,7.0,5.0,65.0,82.0,24.0,99.0,4.0,1.0,1.0,1.0,2.0,2.0,5.0,201.0,NaN,27:05,3.0,11.0,0.0,1.0,0,0,0
14831,chi-gnb-2011-01-23,2010,20.0,gnb-2010,2011-01-23,gnb,201,chi,Sun,W,21.0,14.0,17.0,30.0,236.0,0.0,2.0,1.0,8.0,8.0,7.0,56.0,54.0,32.0,120.0,3.0,2.0,0.0,0.0,3.0,3.0,8.0,334.0,NaN,34:04,2.0,11.0,0.0,0.0,0,0,1
13829,cin-kan-2008-12-28,2008,17.0,kan-2008,2008-12-28,kan,200,cin,Sun,L,6.0,16.0,19.0,36.0,189.0,1.0,0.0,1.0,2.0,5.0,5.0,52.0,77.0,15.0,31.0,2.0,0.0,0.0,0.0,0.0,0.0,6.0,283.0,NaN,22:21,4.0,13.0,0.0,1.0,0,0,0
14460,nwe-nyj-2009-11-22,2009,11.0,nyj-2009,2009-11-22,nyj,200,nwe,Sun,L,14.0,31.0,8.0,21.0,122.0,1.0,4.0,2.0,14.0,6.0,5.0,38.0,34.0,26.0,104.0,4.0,0.0,0.0,0.0,2.0,2.0,5.0,213.0,NaN,20:06,2.0,10.0,1.0,1.0,0,0,0
10864,oti-was-2002-10-06,2002,5.0,oti-2002,2002-10-06,oti,200,was,Sun,L,14.0,31.0,24.0,39.0,230.0,1.0,3.0,4.0,15.0,6.0,5.0,61.0,54.0,15.0,59.0,3.0,1.0,0.0,0.0,2.0,2.0,5.0,223.0,NaN,25:48,2.0,11.0,1.0,2.0,1,0,0
17377,car-was-2015-11-22,2015,11.0,car-2015,2015-11-22,car,201,was,Sun,W,44.0,16.0,21.0,34.0,226.0,5.0,0.0,2.0,20.0,7.0,6.0,61.0,120.0,39.0,142.0,3.0,0.0,3.0,3.0,5.0,5.0,4.0,207.0,NaN,38:07,7.0,14.0,0.0,1.0,1,0,1


Okay, almost there. Our ties are coming through, but we get NaNs for teams that haven't tied, so we'll need to replace those with zeros.

In [19]:
# add 'margin' col
margins = game_df.pts_off - game_df.pts_def
game_df.insert(loc=11, column='margin', value=margins)
game_df[mask].head()

,game_id,season_year,week_num,team_year,full_game_date,team,decade,opp,game_day_of_week,game_outcome,pts_off,margin,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,overtime,time_of_poss,third_down_success,third_down_att,fourth_down_success,fourth_down_att,home,result_tie,result_win
155,gnb-tam-1980-10-12,1980,6.0,gnb-1980,1980-10-12,gnb,198,tam,Sun,T,14.0,0.0,14.0,35.0,51.0,415.0,1.0,2.0,1.0,3.0,8.0,8.0,68.0,83.0,47.0,154.0,3.0,1.0,0.0,3.0,2.0,2.0,8.0,297.0,OT,NaN,NaN,NaN,NaN,NaN,0,1,0
422,gnb-tam-1980-10-12,1980,6.0,tam-1980,1980-10-12,tam,198,gnb,Sun,T,14.0,0.0,14.0,6.0,25.0,82.0,0.0,0.0,3.0,21.0,4.0,2.0,24.0,40.0,35.0,180.0,5.0,1.0,0.0,0.0,2.0,2.0,11.0,482.0,OT,NaN,NaN,NaN,NaN,NaN,1,1,0
669,mia-nyj-1981-10-04,1981,5.0,mia-1981,1981-10-04,mia,198,nyj,Sun,T,28.0,0.0,28.0,21.0,34.0,301.0,2.0,0.0,1.0,5.0,9.0,8.0,61.0,110.0,29.0,98.0,3.0,2.0,0.0,2.0,4.0,4.0,8.0,335.0,OT,NaN,NaN,NaN,NaN,NaN,1,1,0
752,mia-nyj-1981-10-04,1981,5.0,nyj-1981,1981-10-04,nyj,198,mia,Sun,T,28.0,0.0,28.0,28.0,39.0,304.0,4.0,1.0,2.0,6.0,7.0,7.0,71.0,117.0,51.0,242.0,4.0,0.0,0.0,2.0,4.0,4.0,6.0,269.0,OT,NaN,NaN,NaN,NaN,NaN,0,1,0
1027,clt-gnb-1982-12-19,1982,15.0,gnb-1982,1982-12-19,gnb,198,clt,Sun,T,20.0,0.0,20.0,17.0,35.0,224.0,1.0,1.0,4.0,32.0,7.0,5.0,48.0,66.0,40.0,138.0,3.0,1.0,2.0,3.0,2.0,2.0,7.0,259.0,OT,NaN,NaN,NaN,NaN,NaN,0,1,0


In [ ]:
# drop rows for bye weeks
game_df.dropna(axis=0, how='any', subset=['game_outcome'], inplace=True)


# drop exp_pts cols
game_df.drop(['exp_pts_off', 'exp_pts_def', 'exp_pts_st'], axis=1, inplace=True)

In [ ]:
# get integer week_num col

def apply_season_week_col(row):
    week_val = row['week_num']
    if week_val.isdigit():
        return int(week_val)
    else:
        return 0
    
game_df['season_week'] = game_df.apply(apply_season_week_col, axis=1)
game_df

In [ ]:
# add 'playoff_game' column
game_df['playoff_game'] = game_df['season_week'] == 0
game_df

In [ ]:
# add 'prev_week' cols

cols_to_shift = ['wins', 'losses', 'season_week', 'ties', 'pts_off', 'pts_def',
       'margin', 'first_down_off', 'yards_off', 'pass_yds_off', 'rush_yds_off',
       'to_off', 'first_down_def', 'yards_def', 'pass_yds_def', 'rush_yds_def',
       'to_def', 'result_tie', 'result_win']

for col in cols_to_shift:
    new_col = 'prev_' + col
    game_df[new_col] = game_df.groupby('team_year')[col].apply(lambda grp: grp.shift(1))

In [ ]:
# add 'off_bye' (boolean) col
    
def apply_off_bye(row):
    if row['season_week'] > 1: # playoff games have week_num = 0
        off_bye = row['season_week'] - row['prev_season_week'] == 2
    else:
        off_bye = False
    if off_bye == True:
        return 1
    return 0


game_df['off_bye'] = game_df[['season_week', 'prev_season_week']].apply(apply_off_bye, axis=1)



In [ ]:
mask = game_df.team_year == 'chi-2019'
game_df[mask]

In [ ]:
game_df.columns

In [ ]:
# create rolling three feature for these stats:
# using margin instead of wins here... if we want to get num wins in last three weeks,
#  will need to do differently using sum instead of mean

# don't use prev_ columns, we will just shift them down when we use .transform()
roll_cols = ['result_win', 'result_tie', 'pts_off', 'pts_def', 'margin', 'first_down_off', 'yards_off', 
             'pass_yds_off', 'rush_yds_off', 'to_off', 'to2_off', 'yards_def', 'pass_yds_def',
             'rush_yds_def', 'to_def', 'to2_def']

In [ ]:
roll3_cols = ['roll3_' + col_name for col_name in roll_cols]

game_df[roll3_cols] = (game_df.groupby('team_year')[roll_cols]
                      .transform(lambda x: round(x.shift(1).rolling(3).mean(), 3)))


In [ ]:
# get rolling wins (sum) (commented out to try using mean)

rolling_wins = (game_df.groupby('team_year')['result_win']
                      .transform(lambda x: round(x.shift(1).rolling(3).sum(), 3)))

game_df.insert(loc=53, column='roll3_num_wins', value=rolling_wins)

rolling_ties = (game_df.groupby('team_year')['result_win']
                      .transform(lambda x: round(x.shift(1).rolling(3).sum(), 3)))

game_df.insert(loc=53, column='roll3_num_ties', value=rolling_ties)

Now let's add more columns with **Expontentially Weighted Moving Averages (EWMA)** of our stats. This is similar to taking a rolling mean of a team's previous 6 performances, but EWMA weights the most recent game in the window much higher than the week before that, and so on.

For this, we will use a 3-16 week span (greedy). My hypothesis is that the most recent performances are the best predictors, and this should help further quantify recent performance.

In [ ]:
# add ewma cols

ewma_cols = ['ewma_' + col_name for col_name in roll_cols]

game_df[ewma_cols] = (game_df.groupby('team_year')[roll_cols]
                      .transform(lambda x: round(x.shift(1).ewm(span=16, min_periods=3).mean(), 3)))


In [ ]:
mask = game_df.team_year == 'nor-2014'
game_df[mask][['week_num', 'team', 'pass_yds_off', 'prev_pass_yds_off', 'ewma_pass_yds_off', 'roll3_pass_yds_off']].head(18)

In [ ]:
np.mean([197, 120, 208])

Now, let's delete the first three rows of each team-year. We have to do this because we will be using at least some rolling columns for our model.

In [ ]:
game_df.dropna(axis=0, how="any", subset=["roll3_pts_off"], inplace=True)

Looks good!!

**Now, we have to solve a pretty complex problem.**

Currently, each row has all the stats needed for the team in the 'team' column. However, we don't have the same information for the opponent in the same row. Let's make that happen.

To further clarify, for each row, we need to pull the opposing team's rolling stats, adding them back into the same row. This way, our model will only need one row to make a prediction.

Let's take a look at an example.

In [ ]:
mask = (game_df.game_id == 'chi-rai-2019-10-06')
game_df[mask]


So, in the above table, we need to get all of the roll3_x and ewma_x columns from the adjacent tables, respectively. We'll prefix these with opp_.

In [ ]:
game_df.columns

In [ ]:
opp_pull_cols = ['game_id', 'team', 'opp', 'prev_wins', 'prev_losses',
       'prev_ties', 'roll3_result_win', 'roll3_num_wins', 'roll3_result_tie', 
       'roll3_num_ties', 'prev_pts_off', 'prev_pts_def',
       'prev_margin', 'prev_first_down_off', 'prev_yards_off',
       'prev_pass_yds_off', 'prev_rush_yds_off', 'prev_to_off',
       'prev_first_down_def', 'prev_yards_def', 'prev_pass_yds_def',
       'prev_rush_yds_def', 'prev_to_def', 'prev_result_tie',
       'prev_result_win', 'roll3_pts_off', 'roll3_pts_def', 'roll3_margin',
       'roll3_first_down_off', 'roll3_yards_off', 'roll3_pass_yds_off',
       'roll3_rush_yds_off', 'roll3_to_off', 'roll3_yards_def',
       'roll3_pass_yds_def', 'roll3_rush_yds_def', 'roll3_to_def', 'ewma_result_win',
       'ewma_result_tie', 'ewma_num_wins', 'ewma_num_ties', 
       'ewma_pts_off', 'ewma_pts_def', 'ewma_margin', 'ewma_first_down_off',
       'ewma_yards_off', 'ewma_pass_yds_off', 'ewma_rush_yds_off',
       'ewma_to_off', 'ewma_yards_def', 'ewma_pass_yds_def',
       'ewma_rush_yds_def', 'ewma_to_def']

In [ ]:
# # convert all cols to float

# game_df[opp_pull_cols] = game_df[opp_pull_cols].fillna(0)
# for col in opp_pull_cols[3:]:
#     game_df[col] = game_df[col].astype(float)

**Let's solve this using `pd.merge`, using a self-join.**

In [ ]:
game_df = (game_df
             .merge(right=game_df[opp_pull_cols],
                    left_on=['game_id', 'team'],
                    right_on=['game_id', 'opp'],
                    suffixes=[None, '_opp']))

Let's make sure it worked by checking a couple of random game_ids.

In [ ]:
mask = (game_df.game_id == 'dal-nyg-2019-11-04')

game_df[mask][['game_id', 'team', 'opp', 'prev_rush_yds_off',
                 'prev_rush_yds_off_opp', 'roll3_margin', 'roll3_margin_opp']]

In [ ]:
mask = (game_df.game_id == 'chi-rai-2019-10-06')
game_df[mask][['game_id', 'team', 'opp', 'prev_rush_yds_off',
                 'prev_rush_yds_off_opp', 'roll3_margin', 'roll3_margin_opp']]

In [ ]:
mask = (game_df.game_id == 'chi-gnb-1960-12-04')
game_df[mask][['game_id', 'team', 'opp', 'prev_rush_yds_off',
                 'prev_rush_yds_off_opp', 'roll3_margin', 'roll3_margin_opp']]

## Now let's take another look to make sure everything looks good.

If it does, we're ready to start modeling!

In [ ]:
mask = game_df['team_year'] == 'chi-2019'
game_df[mask]

Looks great. Now it's time to start modeling our data! Head over to `model_exploration.ipynb` to continue following along.

In [ ]:
game_df

In [ ]:
game_df.drop_duplicates(subset=['game_id'], inplace=True)

In [ ]:
game_df.shape